In [ ]:
import matplotlib.ticker
from scripts.GetSatellitesGSM import get_artemis_gsm
from scipy.stats import linregress, pearsonr
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import datetime as dt
import pandas as pd
import numpy as np
import os
# Plots all three events with the 4-hour prerun interval. It makes this interval dashed.

In [ ]:
# Plots potential solar wind candidates

os.makedirs('plots/candidates_from_filtered', exist_ok=True)

keys = ['BX_GSM','BY_GSM','BZ_GSM','VX','VY','VZ','N','T']
names = [r'$B_x$ GSM (nT)', r'$B_y$ GSM (nT)', r'$B_z$ GSM (nT)', r'$V_x$ GSM (km/s)', r'$V_y$ GSM (km/s)', r'$V_z$ GSM (km/s)', r'Proton Density (n/cc)', 'Temperature (K)']

omniFileList = sorted(filter(lambda x: os.path.isfile(os.path.join('candidate_solar_wind/from_filtered_omni/omni', x)), os.listdir('candidate_solar_wind/from_filtered_omni/omni')))
artemisFileList = sorted(filter(lambda x: os.path.isfile(os.path.join('candidate_solar_wind/from_filtered_omni/artemis', x)), os.listdir('candidate_solar_wind/from_filtered_omni/artemis')))

file_pairs = [(os.path.join('candidate_solar_wind/from_filtered_omni/artemis', artemis_file), os.path.join('candidate_solar_wind/from_filtered_omni/omni', omni_file)) for artemis_file, omni_file in zip(artemisFileList, omniFileList) if not artemis_file.startswith('.') and not 'copy' in artemis_file]

for filename in file_pairs:
    print(f"Processing file: {filename}")
    artemis_data = pd.read_csv(filename[0], delimiter=',', header=0)
    omni_data = pd.read_csv(filename[1], delimiter=',', header=0)

    artemis_data['Time'] = pd.to_datetime(artemis_data['Time'], format='%Y-%m-%d %H:%M:%S')
    omni_data['Time'] = pd.to_datetime(omni_data['Time'], format='%Y-%m-%d %H:%M:%S')

    x_start = artemis_data['Time'].iloc[0]+dt.timedelta(minutes=30)
    x_end = artemis_data['Time'].iloc[-1]

    artemis_truncated = artemis_data[(artemis_data['Time'] >= x_start) & (artemis_data['Time'] <= x_end)].drop(columns=['Unnamed: 0']).reset_index(drop=True)
    omni_truncated = omni_data[(omni_data['Time'] >= x_start) & (omni_data['Time'] <= x_end)].drop(columns=['Unnamed: 0']).reset_index(drop=True)
    #artemis_truncated = artemis_data.drop(columns=['Unnamed: 0']).reset_index(drop=True)
    #omni_truncated = omni_data.drop(columns=['Unnamed: 0']).reset_index(drop=True)

    fig, ax = plt.subplots(len(keys), 1, figsize=(12, 2*len(keys)), sharex=True)

    for i, (k, n) in enumerate(zip(keys, names)):
        ax[i].plot(artemis_truncated['Time'], artemis_truncated[k], label='Artemis')
        ax[i].plot(omni_truncated['Time'], omni_truncated[k], label='Omni')
        ax[i].set_xlim([x_start, x_end])
        ax[i].set_ylabel(n)
        ax[i].grid(visible=True, which='both', axis='both')

    ax[3].set_ylim([-700, -400])
    ax[3].yaxis.set_major_locator(ticker.MultipleLocator(100))
    ax[3].yaxis.set_minor_locator(ticker.MultipleLocator(50))

    ax[-1].xaxis.set_major_locator(mdates.HourLocator(interval=1))
    ax[-1].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax[-1].xaxis.set_minor_locator(ticker.AutoMinorLocator(4))

    ax[-1].set_xlabel('Time UTC')

    ax[0].set_title(f'Event on {x_start.strftime("%Y-%m-%d")}')
    plt.legend(loc='lower left')
    plt.tight_layout()
    plt.savefig(f'plots/candidates_from_filtered/{x_start.strftime("%Y-%m-%d")}.png', dpi=300)
    plt.close()

In [ ]:
# Plots final solar wind selected, unshifted

os.makedirs('plots/final_candidates', exist_ok=True)

keys = ['BX_GSM','BY_GSM','BZ_GSM','VX','VY','VZ','N','T']
names = [r'$B_x$ GSM (nT)', r'$B_y$ GSM (nT)', r'$B_z$ GSM (nT)', r'$V_x$ GSM (km/s)', r'$V_y$ GSM (km/s)', r'$V_z$ GSM (km/s)', r'Proton Density (n/cc)', 'Temperature (K)']


omniFileList = sorted(filter(lambda x: os.path.isfile(os.path.join('processed_solar_wind/omni', x)), os.listdir('processed_solar_wind/omni')))
artemisFileList = sorted(filter(lambda x: os.path.isfile(os.path.join('processed_solar_wind/artemis', x)), os.listdir('processed_solar_wind/artemis')))
omni_dict = {omni_file.replace('omni_', ''): os.path.join('processed_solar_wind/omni', omni_file)
             for omni_file in omniFileList
             if not omni_file.startswith('.')}
file_pairs = []
for artemis_file in artemisFileList:
    if artemis_file.startswith('.'):
        continue
    # Get the date part by removing 'artemis_' prefix
    date_part = artemis_file.replace('artemis_', '')
    # If we have matching files, create the triplet
    if date_part in omni_dict:
        artemis_path = os.path.join('processed_solar_wind/artemis', artemis_file)
        file_pairs.append((artemis_path, omni_dict[date_part]))

for filename in file_pairs:
    print(f"Processing file: {filename}")
    artemis_data = pd.read_csv(filename[0], delimiter=',', header=0)
    omni_data = pd.read_csv(filename[1], delimiter=',', header=0)

    artemis_data['Time'] = pd.to_datetime(artemis_data['Time'], format='%Y-%m-%d %H:%M:%S')
    omni_data['Time'] = pd.to_datetime(omni_data['Time'], format='%Y-%m-%d %H:%M:%S')

    x_start = artemis_data['Time'].iloc[0]+dt.timedelta(minutes=30)
    x_end = artemis_data['Time'].iloc[-1]

    artemis_truncated = artemis_data[(artemis_data['Time'] >= x_start) & (artemis_data['Time'] <= x_end)].drop(columns=['Unnamed: 0']).reset_index(drop=True)
    omni_truncated = omni_data[(omni_data['Time'] >= x_start) & (omni_data['Time'] <= x_end)].drop(columns=['Unnamed: 0']).reset_index(drop=True)

    fig, ax = plt.subplots(len(keys), 1, figsize=(12, 2*len(keys)), sharex=True)

    for i, (k, n) in enumerate(zip(keys, names)):
        ax[i].plot(artemis_truncated['Time'], artemis_truncated[k], label='Artemis')
        ax[i].plot(omni_truncated['Time'], omni_truncated[k], label='Omni')
        ax[i].set_xlim([x_start, x_end])
        ax[i].set_ylabel(n)
        ax[i].grid(visible=True, which='both', axis='both')

    ax[3].set_ylim([-700, -400])
    ax[3].yaxis.set_major_locator(ticker.MultipleLocator(100))
    ax[3].yaxis.set_minor_locator(ticker.MultipleLocator(50))

    ax[-1].xaxis.set_major_locator(mdates.HourLocator(interval=3))
    ax[-1].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax[-1].xaxis.set_minor_locator(mdates.HourLocator(interval=1))

    ax[-1].set_xlabel('Time UTC')

    ax[0].set_title(f'Event on {x_start.strftime("%Y-%m-%d")}')
    plt.legend(loc='lower left')
    plt.tight_layout()
    plt.savefig(f'plots/final_candidates/{x_start.strftime("%Y-%m-%d")}.png', dpi=300)
    plt.close()

In [ ]:
keys = ['BX_GSM','BY_GSM','BZ_GSM','VX','VY','VZ','N','T']
names = [r'$B_x$ GSM (nT)', r'$B_y$ GSM (nT)', r'$B_z$ GSM (nT)', r'$V_x$ GSM (km/s)', r'$V_y$ GSM (km/s)', r'$V_z$ GSM (km/s)', r'Proton Density (n/cc)', 'Temperature (K)']

omniFileList = sorted(filter(lambda x: os.path.isfile(os.path.join('candidate_solar_wind/final/omni', x)), os.listdir('candidate_solar_wind/final/omni')))
artemisFileList = sorted(filter(lambda x: os.path.isfile(os.path.join('candidate_solar_wind/final/artemis', x)), os.listdir('candidate_solar_wind/final/artemis')))

file_pairs = [(os.path.join('candidate_solar_wind/final/artemis', artemis_file), os.path.join('candidate_solar_wind/final/omni', omni_file)) for artemis_file, omni_file in zip(artemisFileList, omniFileList) if not artemis_file.startswith('.') and not 'copy' in artemis_file]

for filename in file_pairs:
    # File importing and processing ------------------------------
    print(f"Processing file: {filename}")
    artemis_data = pd.read_csv(filename[0], delimiter=',', header=0)
    omni_data = pd.read_csv(filename[1], delimiter=',', header=0)

    artemis_data['Time'] = pd.to_datetime(artemis_data['Time'], format='%Y-%m-%d %H:%M:%S')
    omni_data['Time'] = pd.to_datetime(omni_data['Time'], format='%Y-%m-%d %H:%M:%S')

    x_start = artemis_data['Time'].iloc[0]+dt.timedelta(minutes=30)
    x_end = artemis_data['Time'].iloc[-1]

    artemis_truncated = artemis_data[(artemis_data['Time'] >= x_start) & (artemis_data['Time'] <= x_end)].drop(columns=['Unnamed: 0']).reset_index(drop=True)
    omni_truncated = omni_data[(omni_data['Time'] >= x_start) & (omni_data['Time'] <= x_end)].drop(columns=['Unnamed: 0']).reset_index(drop=True)
    # -------------------------------------------------------------

    fig1, ax1 = plt.subplots(len(keys), 1, figsize=(12, 2*len(keys)), sharex=True)
    for i, (k, n) in enumerate(zip(keys, names)):
        ax1[i].plot(artemis_truncated['Time'], artemis_truncated[k], label='Artemis')
        ax1[i].plot(omni_truncated['Time'], omni_truncated[k], label='Omni')
        ax1[i].set_xlim([x_start, x_end])
        ax1[i].set_ylabel(n)
        ax1[i].grid(visible=True, which='both', axis='both')

    ax1[2].set_ylim([-700, -400])
    ax1[2].yaxis.set_major_locator(ticker.MultipleLocator(100))
    ax1[2].yaxis.set_minor_locator(ticker.MultipleLocator(50))
    ax1[3].legend(loc='lower left')
    ax1[4].set_ylim([0, 5])
    ax1[4].set_ylabel(r'Parameter Fluctuation')
    ax1[4].grid(visible=True, which='both', axis='both')

    ax1[-1].xaxis.set_major_locator(mdates.HourLocator(interval=1))
    ax1[-1].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax1[-1].xaxis.set_minor_locator(ticker.AutoMinorLocator(4))
    ax1[-1].set_xlabel('Time UTC')
    ax1[0].set_title(f'Event on {x_start.strftime("%Y-%m-%d")}')

    plt.legend(loc='lower left')
    plt.tight_layout()
    plt.savefig(f'plots/final_candidates/withdeltas/{x_start.strftime("%Y-%m-%d")}.png', dpi=300)
    plt.close()